In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
input_img = 'Ag-Bi.PNG'

img2 = cv.imread(input_img,0)
img2 = cv.medianBlur(img2,3)
ret,th1 = cv.threshold(img2,250,265,cv.THRESH_BINARY)
th1 = cv.medianBlur(th1,3)

th2 = cv.adaptiveThreshold(img2,255,cv.ADAPTIVE_THRESH_MEAN_C,\
            cv.THRESH_BINARY,11,2)
th3 = cv.adaptiveThreshold(img2,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,11,2)
titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img2, th1, th2, th3]

figure (figsize=(20,20))
for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

error: ignored

In [ ]:
img3 = cv.imread(input_img,0)
img3 = cv.medianBlur(img2,3)
ret,th1 = cv.threshold(img2,245,255,cv.THRESH_BINARY)


num_labels, labels_im, stats_im, centroid_im  = cv.connectedComponentsWithStats(th1)

def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv.cvtColor(labeled_img, cv.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    #cv.imshow('labeled.png', labeled_img)
    figure (figsize=(15,15))
    plt.subplot(121)
    plt.imshow(img2,cmap = 'gray')
    plt.title('th1 blurred threshold'), plt.xticks([]), plt.yticks([])

    figure (figsize=(15,15))
    plt.subplot(121)
    plt.imshow(th1,cmap = 'gray')
    plt.title('th1 blurred threshold'), plt.xticks([]), plt.yticks([])

    figure (figsize=(15,15))
    plt.subplot(122)
    plt.imshow(labeled_img,cmap = 'gray')
    plt.title('contours'), plt.xticks([]), plt.yticks([])

    cv.waitKey()


imshow_components(labels_im)

In [ ]:
#find out what region of the graph the input is in
def phase_finder(y,x):
  iter = 0
  phase = labels_im[y,x]
  #if there are less than 50 pixels for an area it is excluded and the input is moved to a nearby value, it no nearby value works an error message is spit out
  while stats_im[phase,4] < 50:
    x = x+1
    y = y+1
    iter = iter+1
    if iter > 10:
      print('there was an error with the location selected please try again')
      return()
  return(phase)

In [ ]:
#placeholder until the program is linked to the input
input_x = 49
input_y = 472
input = [input_y,input_x]

phase = phase_finder(input_y, input_x)

In [ ]:
#finds the min and max values for the x and y of the area

#finding max and min values for y
y_distribution = (np.count_nonzero(labels_im == phase, axis=1))

#finding max and min values for x
x_distribution = (np.count_nonzero(labels_im == phase, axis=0))

#finds the min and max values in the x and y direction for the specific phase
ls_x = [i for i, e in enumerate(x_distribution) if e !=0]
x_min = ls_x[0]
x_max = ls_x[len(ls)-1]

ls_y = [i for i, e in enumerate(y_distribution) if e !=0]
y_min = ls_y[0]
y_max = ls_y[len(ls_y)-1]
